In [61]:
import os,re,sys
import pandas as pd
import numpy as np
import subprocess
import pickle
import shutil
import mutagen
import requests

In [3]:
"BBDown  -M '<videoTitle>#[P<pageNumberWithZero>]#<pageTitle>#<publishDate>#<bvid>' "

data = pd.read_excel('eoe_files.xlsx')

In [4]:
member = ('露早' ,'莞儿', '柚恩', '虞莫','米诺')
patt = '|'.join([i[0]+'[^露莞柚虞米]' for  i in member]) + '|[&＆]'
def get_members(s):
    x = [i for i in member if i[0] in s ]  # 第一个字
    if len(x): s = re.sub('EOE\s?|原$','',s)
    if '生贺' in s : s= '生贺'
    elif '原创' in s : s = '原创'
    x ='&'.join(x)
    return x , x + re.sub(patt, '', s)

In [5]:
pkldata = pickle.load(open('录播bilix_info.pkl','rb'))

In [6]:
cols = ['bv', 'singer','singer0','song','name', 'is_single','is_sing','ori']
data_list = []

for name in data.name:
    ori = name
    name = name.split('#')[0]   
    is_single = 1 if 'mp4' in name else 0
    name = re.sub('\[.*mp4$','',name)
    bv = re.sub('.mp4$','',ori).split('#')[-1]
    bv = np.nan if is_single else bv

    if re.findall('[翻合]唱|[团金歌]曲|原[唱创]|混剪',name) : is_sing = 1 
    elif re.findall('舞|跳|摇|蹦[蹦迪]',name): is_sing = 2 
    else: is_sing = 0
        
    singer = re.findall('【(.*?)】', re.sub('【EOE一周年】|【新衣特别晚会】','',name))
    singer = re.sub('[翻跳唱|丨｜精译]|生日会[_]?|202\d|组合|一周年' ,'', singer[-1] ) if singer else ''
    singer,singer0 = get_members(singer) 
    if not singer: singer = get_members(name)[0]
    if not singer : singer = 'EOE' if re.findall('EOE|合唱|团',name) else ''
    song = re.findall('[《『](.*?)[』》]', name)
    song = song[0] if song else ''
    data_list.append((bv, singer,singer0,song,name, is_single,is_sing,ori))

res = pd.DataFrame(data_list, columns= cols) 

In [7]:
bvs = []
cols2 = ['bv', 'name', 'aid', 'ct','desc' ]
for bv,bdata in pkldata.items():
    bvs.append((bv, bdata.title, bdata.aid, len(bdata.pages) ,bdata.desc))
    
pkdf = pd.DataFrame(bvs ,columns=cols2)
fillbv = pd.merge(res[res.bv.isna()],  pkdf[pkdf.ct==1].reset_index(drop=True) 
                    ,on = 'name', how = 'left')

In [8]:
res['name_ori'] = res['name']
namask = res.bv.isna()
res.loc[namask,'name'] = res.loc[namask,'name'].str.replace('_','')
#下载时有下划线?          输出重复的
pkdf[pkdf.ct ==1 & pkdf.duplicated('name')].to_excel('duplicate.xlsx')
fillbv = pd.merge(res[namask],pkdf[pkdf.ct ==1].drop_duplicates(subset='name').reset_index(drop=True)
                  ,on= 'name',how='left')

res.loc[namask,'bv'] = fillbv['bv_y'].tolist()  # loc 会按index赋值


In [9]:
singer_mask = res.singer.str.len() == 0
fill_singer = pd.merge(res[singer_mask],pkdf.reset_index(drop=True)
                  ,on= 'bv',how='left')
 
def get_members2(s): 
    x = [i for i in member if i[0] in s ]  # 第一个字 
    x ='&'.join(x) if x else '' 
    if (not x ): x = 'EOE' 
    return x  

fill_singer['sin2'] = fill_singer.desc.fillna('').apply(lambda x : get_members2(  re.findall('.*?[-]{4,}',x ,flags=re.S)[0]  if re.findall('.*?[-]{4,}',x ,flags=re.S) else '' ) )
res.loc[singer_mask,'singer'] = fill_singer['sin2'].tolist()

In [138]:
res.to_excel('res.xlsx') # 检查name重复 bv空的 

res = pd.read_excel('res-modify.xlsx')

In [139]:
res['folder0'] = res['singer'].apply(lambda x:x if len(x) else '空')
sing_map = {0:'创作剪辑',1:'唱',2:'跳'}
res['folder1'] = res['is_sing'].map(sing_map)

count_map = res.bv.value_counts()
res['bvct'] = res.bv.map(count_map)
res['file'] = res.ori
res.loc[res.bvct == 1, 'ori'] = res.loc[res.bvct == 1, 'name_ori'] 


In [140]:
split = res.ori.str.split('#',expand=True).iloc[:,:3]
split.columns = ['title','pn','ptitle']
res = res.join(split)
res.to_excel('res.xlsx',index=None) # 检查 

##### mid 及歌曲信息

In [ ]:
#  https://api.bilibili.com/x/player/v2?cid=875796755&aid=562069714   #  .data.bgm_info.music_id
 # https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA407125921754034458
import asyncio 
from bilix.sites.bilibili import api
from httpx import AsyncClient

async def main(id): 
    client = AsyncClient(**api.dft_client_settings)
    data = await api._get_video_basic_info_from_api(client, f'https://www.bilibili.com/video/{id}') 
    return data 
# await main('BV1us421N7aJ')

dict_info = {} 
errbv = []
for i,bid  in enumerate( res.bv.unique()):  
    try:
        info = await main(bid)
        print(i,':',bid) 
    except Exception as e:
        errbv.append(bv)
        print(i,bv,'获取信息出错',e)
        continue 
    bv,title = info.bvid, info.title
    pn = ''
    dict_info[bv] = info 

print(errbv)

with open(f'带有cid.pkl', 'wb') as f:
    pickle.dump(dict_info, f)   # pkl 存储
cidata = pickle.load(open('带有cid.pkl','rb'))

bvs = []
cols3 = ['bv',  'aid', 'pn', 'cid' ]
for bv,bdata in cidata.items(): 
    cidnfo = []
    for p in bdata.pages: 
        pn = re.findall('\?p=(\d+)',p.p_url)
        pn = int(pn[0]) if pn else 1
        cidnfo.append((bv, bdata.aid, pn, p.pcid))
   
    bvs.extend(cidnfo) 

cid_pd = pd.DataFrame(bvs,columns= cols3)


In [141]:

res['pn2'] =  res.pn.str.extract('P(\d+)').fillna(1).astype(int) 
aid_cid = pd.merge(res, cid_pd , left_on=['bv','pn2'],right_on=['bv','pn'] ,how='left') #.to_excel('test.xlsx') # aid pn_y cid,  
aid_cid.aid = aid_cid.aid.fillna(1854252190) # 想你时风起
aid_cid.cid = aid_cid.cid.fillna(1538566546) # 1854252190, cid=1538566546
url = 'https://api.bilibili.com/x/player/v2?cid=875796755&aid=562069714'   #  .data.bgm_info.music_id
# https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA407125921754034458


In [ ]:

def get_mid(cid, aid):
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://api.bilibili.com/x/player/v2?cid={cid}&aid={aid}' 
    mid='' 

    reqres = requests.get(url, headers=headers) 
    if reqres.status_code == 200:
        data = reqres.json()
        mid = data['data'].get('bgm_info')
        mid =  mid.get('music_id') if mid else ''
        print(cid)
    else: 
        print(f"Failed to get details for aV: {aid},{cid}")
 
    return mid

 
# https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA456146494714591308

def get_mid_info(mid):
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id={mid}'  
    info ={}
    reqres = requests.get(url, headers=headers) 
    if reqres.status_code == 200:
        data = reqres.json()
        info = data['data'] 
    else: 
        print(f"Failed to get details for mid: {mid}")
 
    return info



In [ ]:
mids = []
for i in aid_cid.itertuples():
    mid = get_mid( int(i.cid) , int(i.aid) )
    mids.append(mid)

aid_cid['mid'] = mids
mid_info = {} 
m2 = set(mids) 
m2.remove('') 
 
for i,mid in enumerate(m2):
    minfo = get_mid_info(mid) 
    mid_info[mid] = minfo
    print(i,mid)
 
with open(f'midinfo.pkl', 'wb') as f:
    pickle.dump(mid_info, f)  


In [145]:
music_df = pd.DataFrame(mid_info).T.reset_index() 
music_df.drop(columns=['hot_song_heat','ab_test','music_comment'] ,inplace = True)
final_merge = pd.merge(aid_cid,  music_df , left_on = 'mid', right_on = 'index', how = 'left')

In [152]:
final_merge.rename(columns = {'pn_x':'pn'} ,inplace = True)

#### 文件部分

In [154]:
oripath = 'origins/'
files_path = []
from_path = []
final_merge.pn = final_merge.pn.fillna('')
final_merge.ptitle = final_merge.ptitle.fillna('')

for row in final_merge.itertuples():
    f0 = 'EOE' if row.folder0.count('&')>1 else row.folder0
    files_path.append('/'.join([i for i in 
                                [f0, row.folder1 , row.title, row.pn + row.ptitle] if i]) +'mp4')
    from_path.append( oripath  + row.file)

In [179]:
folders = set(os.path.dirname(p) for p in files_path)
for fd in folders:
    os.makedirs(fd , exist_ok=True) 

In [184]:
 
# final_merge.to_excel('final_merge.xlsx') 

In [224]:
final_merge['artist'] = final_merge['singer'].str.replace('&','/')
final_merge['albumartist'] = final_merge['singer'] + final_merge['is_sing'].map( {0:'创',1:'',2:'舞'} )
final_merge['ftitle'] = final_merge.song
final_merge.loc[final_merge.song.isna() , 'ftitle'] = final_merge.ptitle[final_merge.song.isna()]
final_merge.loc[final_merge.ftitle.isna(), 'ftitle'] = final_merge.loc[final_merge.ftitle.isna(), 'music_title'] 
#  str.len()对空的不生效!!!  & final_merge.ftitle.str.len()==0  final_merge.song.isna() 
final_merge.loc[final_merge.ftitle.isna(), 'ftitle'] = final_merge.name[final_merge.ftitle.isna() ] 

final_merge['origin_artist'].fillna('',inplace=True)
final_merge.to_excel('final_merge.xlsx')

In [ ]:
for row in final_merge.itertuples():
    file =  oripath + row.file 
    mfile = mutagen.File(file, easy=True) 

    p1 = '' if row.pn2==1 else '?p=' + str(row.pn2)
    mfile['comment'] = row.bv + p1
    mfile['title'] = row.ftitle
    
    if row.origin_artist:
        mfile['album'] = row.origin_artist
        print(file,row.origin_artist)

    mfile['albumartist'] = row.albumartist
    mfile['artist'] = row.artist
    mfile.save(file)


#### ffmpeg and move

In [273]:
opath = oripath  # 'test/'
cover_path = opath + 'cover/'
audio_path = opath + 'audio/'
out_path = opath + 'out/' 
os.makedirs(cover_path,exist_ok = True)
os.makedirs(audio_path,exist_ok = True)
os.makedirs(out_path,exist_ok = True)

for f in os.listdir( opath ) : # res.file
    if '.mp4' not in f: continue 
    path = opath + f
    img  = f.replace('.mp4','.jpg')
    m4a  = f.replace('.mp4','.m4a')
    imgp = f"{cover_path}{img}"
    m4ap = f"{audio_path}{f}" #m4a
    outp = f"{out_path}{f}"

    # 1st step
    # cmd = f'ffmpeg -i "{path}"  -map 0:v -map -0:V -c copy "{imgp}"  -vn -map_metadata 0 -acodec copy "{m4ap}" -y' 
    # result = subprocess.check_output(cmd).decode('utf-8') 
 
    # 压缩后 2nd step
    cmd2 = f'ffmpeg -i "{m4ap}" -i "{imgp}" -c copy -disposition:v attached_pic -map_metadata 0 "{outp}" -y' 
    result2 = subprocess.check_output(cmd2).decode('utf-8') 
    shutil.move(outp , f"{out_path}{m4a}" ) # 阿里分享音频 要mp4改名为m4a



In [70]:
# videos_link[803] # av832691471'  av235210893'
 #  "--save-archives-to-file" }, "将下载过的视频记录到本地文件中, 用于后续跳过下载同个视频");
#   "--delay-per-page" }, "设置下载合集分P之间的下载间隔时间(单位: 秒, 默认无间隔)");
#  "--file-pattern", "-F" },  

In [ ]:
# 最后移动
for i,fp in enumerate(from_path):
    shutil.move(fp , files_path[i])

#### 下载多出的(一次)

In [ ]:
add_bvs = pd.read_excel('duplicate-modify.xlsx', sheet_name='Sheet3')
# 下载后 文件名字加入有2694个eoe_files.xlsx的后面再进行处理

def run_cmd(cmd):
# 使用Popen创建进程，并与进程进行复杂的交互
    proc = subprocess.Popen(
        cmd, # cmd特定的查询空间的命令  'BBDown av1503004887 --audio-only'
        stdin=None, # 标准输入 键盘
        stdout=subprocess.PIPE, # -1 标准输出（演示器、终端) 保存到管道中以便进行操作
        stderr=subprocess.PIPE, # 标准错误，保存到管道
        shell=False)       #如果使用 shell=True,则需要在命令字符串中处理引号。 #而 shell=False 时,可以直接在 参数列表 中使用引号表示字符串参数
    out, err = proc.communicate() # 获取输出和错误信息
    out = out.decode('gbk')
    err = err.decode('gbk')
    err = '\n'+err if err else ''
    print(f'out:{out[-30:]} {err}') 

 
for i,bv  in enumerate(add_bvs['bvs'].tolist()):
    cmd =  f"BBDown {bv} -F <videoTitle>#[P<pageNumberWithZero>]#<pageTitle>#<publishDate>#<bvid>  -M <videoTitle>#[P<pageNumberWithZero>]#<pageTitle>#<publishDate>#<bvid>  -p 1 "
    print(cmd)
    # run_cmd(cmd) 


In [13]:
#  https://api.bilibili.com/x/player/v2?cid=875796755&aid=562069714   #  .data.bgm_info.music_id
 # https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA407125921754034458
import asyncio 
from bilix.sites.bilibili import api
from httpx import AsyncClient

async def main(id): 
    client = AsyncClient(**api.dft_client_settings)
    data = await api._get_video_basic_info_from_api(client, f'https://www.bilibili.com/video/{id}') 
    return data 
# await main('BV1us421N7aJ')

In [ ]:

dict_info = {} 
errbv = []
for i,bid  in enumerate( res.bv.unique()):  
    try:
        info = await main(bid)
        print(i,':',bid) 
    except Exception as e:
        errbv.append(bv)
        print(i,bv,'获取信息出错',e)
        continue 
    bv,title = info.bvid, info.title
    pn = ''
    dict_info[bv] = info 

print(errbv)

with open(f'带有cid.pkl', 'wb') as f:
    pickle.dump(dict_info, f)   # pkl 存储

In [51]:
cidata = pickle.load(open('带有cid.pkl','rb'))

bvs = []
cols3 = ['bv',  'aid', 'pn', 'cid' ]
for bv,bdata in cidata.items(): 
    cidnfo = []
    for p in bdata.pages: 
        pn = re.findall('\?p=(\d+)',p.p_url)
        pn = int(pn[0]) if pn else 1
        cidnfo.append((bv, bdata.aid, pn, p.pcid))
   
    bvs.extend(cidnfo) 

cid_pd = pd.DataFrame(bvs,columns= cols3)


In [52]:
res['pn2'] =  res.pn.str.extract('P(\d+)').fillna(1).astype(int) 

In [57]:
aid_cid = pd.merge(res, cid_pd , left_on=['bv','pn2'],right_on=['bv','pn'] ,how='left') #.to_excel('test.xlsx') # aid pn_y cid,  
aid_cid.aid = aid_cid.aid.fillna(1854252190) # 想你时风起
aid_cid.cid = aid_cid.cid.fillna(1538566546) # 1854252190, cid=1538566546

In [97]:
url = 'https://api.bilibili.com/x/player/v2?cid=875796755&aid=562069714'   #  .data.bgm_info.music_id
# https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA407125921754034458

def get_mid(cid, aid):
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://api.bilibili.com/x/player/v2?cid={cid}&aid={aid}' 
    mid='' 

    reqres = requests.get(url, headers=headers) 
    if reqres.status_code == 200:
        data = reqres.json()
        mid = data['data'].get('bgm_info')
        mid =  mid.get('music_id') if mid else ''
        print(cid)
    else: 
        print(f"Failed to get details for aV: {aid},{cid}")
 
    return mid

 
# https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id=MA456146494714591308

def get_mid_info(mid):
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://api.bilibili.com/x/copyright-music-publicity/bgm/detail?music_id={mid}'  
    info ={}
    reqres = requests.get(url, headers=headers) 
    if reqres.status_code == 200:
        data = reqres.json()
        info = data['data'] 
    else: 
        print(f"Failed to get details for mid: {mid}")
 
    return info



In [ ]:
mids = []
for i in aid_cid.itertuples():
    mid = get_mid( int(i.cid) , int(i.aid) )
    mids.append(mid)

aid_cid['mid'] = mids

In [98]:
mid_info = {} 
m2 = set(mids) 
m2.remove('') 
 
for i,mid in enumerate(m2):
    minfo = get_mid_info(mid) 
    mid_info[mid] = minfo
    print(i,mid)
 
with open(f'midinfo.pkl', 'wb') as f:
    pickle.dump(mid_info, f)  

In [ ]:
# [ item['music_title'] for mid , item in mid_info.items()]
music_df = pd.DataFrame(mid_info).T.reset_index() 
music_df.drop(columns=['hot_song_heat','ab_test','music_comment'] ,inplace = True)

In [134]:
final_merge = pd.merge(aid_cid,  music_df , left_on = 'mid', right_on = 'index', how = 'left')